<a href="https://colab.research.google.com/github/dbizzaro/NotebooksKRL/blob/main/KRL_modeling_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modeling Exercise

Alice (A), Bob (B), Craig (C), and Donna (D) are four friends that
  want to take a tour with their motorbikes. Everybody can decide
  either to go with somebody else or to ride a bike alone.  Use
  propositional logic to find all the
  possible configurations for the tour.

In [ ]:
from sympy import *


We introduce the following propositional variables:

*   $Rides_X$ for "$X$ drives a bike", for $X\in\{A,B,C,D\}$
*   $Pass_{XY}$ for "$X$ give a pass to $Y$", for $X,Y\in\{A,B,C,D\}$ and $X\neq Y$

In [ ]:
Friends = ['A','B','C','D']
Rides = {x:Symbol(f"Rides {x}") for x in Friends}
Pass = {(X,Y):Symbol(f"Pass {X}{Y}") for X in Friends for Y in Friends if X != Y}

print(Rides)
print(Pass)

{'A': Rides A, 'B': Rides B, 'C': Rides C, 'D': Rides D}
{('A', 'B'): Pass AB, ('A', 'C'): Pass AC, ('A', 'D'): Pass AD, ('B', 'A'): Pass BA, ('B', 'C'): Pass BC, ('B', 'D'): Pass BD, ('C', 'A'): Pass CA, ('C', 'B'): Pass CB, ('C', 'D'): Pass CD, ('D', 'A'): Pass DA, ('D', 'B'): Pass DB, ('D', 'C'): Pass DC}


We add the following axioms:
* Each friend either rides or gets a lift: $$\phi_1 :=  \bigwedge_{X}\left(Rides_X \vee \bigvee_{Y\neq X}Pass_{YX}\right)$$
* If $Y$ gets a lift from $X$, then $X$ rides the bike: $$ \phi_2 := \bigwedge_{X\neq Y}(Pass_{XY}\to Rides_X)$$
* If $X$ rides, then he/she does not get a lift: $$\phi_3 := \bigwedge_{X\neq Y}(Rides_X\to \neg Pass_{YX})$$
* Each $Y$ can get a lift from at most one person: $$\phi_4 := \bigwedge_{X\neq X'\neq Y}(Pass_{XY}\to \neg Pass_{X'Y})$$
* Each $X$ can give a lift to at most one person: $$\phi_5 := \bigwedge_{X\neq Y\neq Y'}(Pass_{XY}\to \neg Pass_{XY'})$$

In [ ]:
phi1 = And(*[Rides[X] | Or(*[Pass[Y,X] for Y in Friends if Y!=X]) for X in Friends])
phi2 = And(*[Pass[X,Y] >> Rides[X] for X in Friends for Y in Friends if X!=Y])
phi3 = And(*[Pass[X,Y] >> ~Rides[Y] for X in Friends for Y in Friends if X!=Y])
phi4 = And(*[Pass[X,Y] >> ~Pass[X1,Y] for X in Friends for X1 in Friends for Y in Friends if X1!=X and Y!=X1 and Y!=X])
phi5 = And(*[Pass[X,Y] >> ~Pass[X,Y1] for X in Friends for Y in Friends for Y1 in Friends if X!=Y and X!=Y1 and Y!=Y1])

We run the algorithm for retrieving all the models of $\phi_1 \land \phi_2 \land \phi_3 \land \phi_4 \land \phi_5$.

In [ ]:
for m in satisfiable(phi1 & phi2 & phi3 & phi4 & phi5, all_models=True):
  print(m)

{Rides B: True, Rides C: True, Rides D: True, Rides A: True, Pass CD: False, Pass CB: False, Pass AB: False, Pass AD: False, Pass BD: False, Pass DA: False, Pass BA: False, Pass CA: False, Pass AC: False, Pass DC: False, Pass DB: False, Pass BC: False}
{Rides B: True, Pass BC: True, Rides D: True, Rides A: True, Pass CD: False, Pass CB: False, Pass AB: False, Pass AD: False, Pass BD: False, Pass DA: False, Pass BA: False, Pass CA: False, Rides C: False, Pass AC: False, Pass DC: False, Pass DB: False}
{Pass DB: True, Rides C: True, Rides D: True, Rides A: True, Pass CD: False, Pass CB: False, Pass AB: False, Pass AD: False, Pass BD: False, Pass DA: False, Pass BC: False, Pass BA: False, Pass CA: False, Pass AC: False, Pass DC: False, Rides B: False}
{Rides B: True, Pass DC: True, Rides D: True, Rides A: True, Pass CD: False, Pass CB: False, Pass AB: False, Pass AD: False, Pass BD: False, Pass DA: False, Pass BA: False, Pass CA: False, Rides C: False, Pass AC: False, Pass DB: False, Pass

In [ ]:
for m in satisfiable(phi1 & phi2 & phi3 & phi4 & phi5, all_models=True):
  list_true = [x for x in m if m[x]]
  print(list_true)

[Rides B, Rides C, Rides D, Rides A]
[Rides B, Pass BC, Rides D, Rides A]
[Pass DB, Rides C, Rides D, Rides A]
[Rides B, Pass DC, Rides D, Rides A]
[Rides B, Pass AC, Rides D, Rides A]
[Pass AC, Pass DB, Rides D, Rides A]
[Rides B, Rides C, Pass CA, Rides D]
[Pass DB, Rides C, Pass CA, Rides D]
[Rides B, Rides C, Pass BA, Rides D]
[Rides B, Pass DC, Pass BA, Rides D]
[Rides B, Rides C, Rides D, Pass DA]
[Rides B, Pass BC, Rides D, Pass DA]
[Rides C, Rides B, Pass BD, Rides A]
[Pass AC, Rides B, Pass BD, Rides A]
[Rides C, Pass CA, Rides B, Pass BD]
[Rides C, Rides B, Pass AD, Rides A]
[Pass BC, Rides B, Pass AD, Rides A]
[Rides C, Rides D, Pass AB, Rides A]
[Pass DC, Rides D, Pass AB, Rides A]
[Rides D, Rides C, Pass CB, Rides A]
[Rides D, Rides C, Pass CB, Pass DA]
[Rides C, Pass CB, Pass AD, Rides A]
[Rides B, Rides C, Pass CD, Rides A]
[Pass BA, Rides B, Rides C, Pass CD]
[Rides C, Pass CD, Pass AB, Rides A]
